インストールパッケージのバージョン確認

In [1]:
import tensorflow as tf
import transformers as tr
print(tf.__version__)
print(tr.__version__)

2.12.0
4.28.0.dev0


In [2]:
!git lfs install
!git clone https://huggingface.co/decapoda-research/llama-7b-hf

Git LFS initialized.
fatal: destination path 'llama-7b-hf' already exists and is not an empty directory.


2023/4/1時点ではHugging Faceに上がっているLLaMAのファイルにバグ(名前変更による修正もれ?)があるため、手動で設定を変更します。

tokenizer_classを`LLaMATokenizer`から`LlamaTokenizer`に変更します。  

 ```bash
 $ vi LargeLanguageModel/llama-7b-hf/tokenizer_config.json
 ```
 ```json
 {"bos_token": "", "eos_token": "", "model_max_length": 1000000000000000019884624838656, "tokenizer_class": "LlamaTokenizer", "unk_token": ""}
 ```

M1 MacのGPUに対応するため

In [2]:
import torch
import tensorflow as tf

device = torch.device("mps")

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig

tokenizer = AutoTokenizer.from_pretrained("./llama-7b-hf/", config="./llama-7b-hf/tokenizer_config.json")
config = AutoConfig.from_pretrained("./llama-7b-hf/", use_metal=True)

model = AutoModelForCausalLM.from_pretrained("./llama-7b-hf/", config=config).to(device)

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [4]:
%%time
import torch

# 生成する文章の入力と生成パラメータの設定
prompt = "今日の晩ご飯は"
length = 30
temperature = 0.8

# トークン化とテンソル変換
input_ids = tokenizer.encode(prompt, return_tensors="pt")
#input_ids = input_ids.to(model.device)
input_ids = input_ids.to(device)

# 生成の実行
output = model.generate(
    input_ids, 
    do_sample=True, 
    max_length=length, 
    temperature=temperature,
)

# 生成された文章のデコード
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

# 生成された文章の出力
print(generated_text)

/Users/master/mambaforge/envs/ml/lib/python3.9/site-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


 今日の晩ご飯はありません。私は、晩飯
CPU times: user 4.5 s, sys: 3min 43s, total: 3min 47s
Wall time: 1h 29min 23s
